## Anthropic APIs
### Messages 
```code
POST / v1 / messages
```
Operate using alternating `user` and `assistant` turns  
Each input message must be an object with a `role` and `content`  
The first message must always be the `user` message  

To include a system prompt, use the top-level __system__ parameter — there is no __"system"__ role for input messages in the Messages API.


### Jupyter Notebooks interface  
In general, it's important to understand that the Jupyter Notebook interface, while very convenient for **iterative development and exploration**, does not automatically persist the full execution state across sessions by default. For long-running jobs or important intermediate results, we explicitly save artifacts to disk.   

We also look into workflow tools like Papermill to parameterize and execute notebooks in a more automated fashion that saves the output.  

### Import library(-ies)

In [17]:
# Source keys from an .env
from dotenv import load_dotenv
# Source operating system resources
import os
# Access libraries for AI  
from openai import OpenAI
# Standard library for API requests
import requests
# Enable working with Unix timestamps
from datetime import datetime
# Enable working with json
import json
# Enable display of html responses
from IPython.display import display, HTML, Markdown, Latex
# Iterate paths and more
from pathlib import Path
# For basic file handling 
import shutil
# Search efficently
import re

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Authenticate

In [18]:
# Load environment variables
load_dotenv()
api_key_pplx = os.getenv('PPLX_API_KEY')
api_key_claude = os.getenv('CLAUDE_API_KEY')

In [38]:
# Anthropic
# curl https://api.anthropic.com/v1/messages --header "x-api-key: YOUR_API_KEY" ...
base_url_claude = ""

In [19]:
# Perplexity
base_url_pplx = "https://api.perplexity.ai"

### Keep it DRY

In [20]:
# Choose model
model = "llama-3-sonar-large-32k-online"

# Create a client
client = OpenAI(api_key=api_key_pplx, base_url=base_url_pplx)

In [21]:
# Make a request
def get_completion(user_prompt, system_role, model):
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_prompt}
    ]

    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    # Print the message content
    return response

In [22]:
# Accomodate common formats
def get_completion_json(system_role, user_prompt, model):
    # Note authentication is configured above

    
    # chat completion without streaming
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"}
    )
                         
    # Print the message content
    return response

In [23]:
# Print the content of the response
def summarize_message(from_the_ai):
    # Summarize the message
    print(f"The {response.choices[0].message.role} role finished \n♢ with \"{response.choices[0].finish_reason}\" at: {datetime.fromtimestamp(response.created)}")
    print(f"The model used was: {response.model}")
    print(f"♢ The message was: \n\n{response.choices[0].message.content}")

In [24]:
# Pretty print HTML response
def pretty_print_html(from_the_ai):
    display(HTML(response.choices[0].message.content))

In [25]:
# Pretty print Markdown response
def pretty_print_markdown(from_the_ai):
    display(Markdown(response.choices[0].message.content))

### Processing files

1. Use the latest models. For perplexity.ai API there is a [list](https://docs.perplexity.ai/docs/model-cards):   
2. Instructions must be clear and specific
3. Instruct at the beginning of the prompt
4. Delimit distinct parts with delimiters and use variables to chunk roles, instructions  

5. Precise, __descriptive__, with sufficient detail

In [ ]:
# Create a basic file handling function
# Previously we import os
# AND import shutil

def copy_item(src, dst):
    try:
        if os.path.isfile(src):
            # Copia o arquivo (Copy the file)
            shutil.copy(src, dst)
        elif os.path.isdir(src):
            # Copia a pasta e seu conteúdo (Copy the folder and its contents)
            shutil.copytree(src, dst)
        else:
            print(f"Item não encontrado: {src}")
            return
        
        print(f"Item copiado com sucesso: {src} -> {dst}")
    except FileExistsError:
        print(f"O item de destino já existe: {dst}")
    except PermissionError:
        print(f"Permissão negada ao copiar o item: {src}")
    except Exception as e:
        print(f"Erro ao copiar o item: {src} - {str(e)}")

def move_item(src, dst):
    try:
        # Move o arquivo ou pasta (Move the file or folder)
        shutil.move(src, dst)
        print(f"Item movido com sucesso: {src} -> {dst}")
    except FileNotFoundError:
        print(f"Item de origem não encontrado: {src}")
    except shutil.Error as e:
        print(f"Erro ao mover o item: {src} - {str(e)}")
    except Exception as e:
        print(f"Erro ao mover o item: {src} - {str(e)}")

def delete_item(item_path):
    try:
        if os.path.isfile(item_path):
            # Remove o arquivo (Remove the file)
            os.remove(item_path)
        elif os.path.isdir(item_path):
            # Remove a pasta e seu conteúdo (Remove the folder and its contents)
            shutil.rmtree(item_path)
        else:
            print(f"Item não encontrado: {item_path}")
            return
        
        print(f"Item excluído com sucesso: {item_path}")
    except PermissionError:
        print(f"Permissão negada ao excluir o item: {item_path}")
    except Exception as e:
        print(f"Erro ao excluir o item: {item_path} - {str(e)}")

In [ ]:
pretty_print_markdown(response)

### Display

In [ ]:
pretty_print_markdown(response)

### Process files

In [ ]:
# Use files as inputs for the notebook
this_cwd = Path.cwd()
print(this_cwd.name)
# vs
get_cwd = os.getcwd()
cwd_name = os.path.basename(get_cwd)
print(cwd_name)

In [ ]:
# pathlib seems cleaner, so 
csv_files = list(this_cwd.glob("*.csv"))

for file_path in csv_files:
    print(file_path)
else:
    print(f"no csvs globbed in {this_cwd.name}")



In [ ]:
# Iterate the files
for item in this_cwd.iterdir():
    print(f"a file: {item.name}")
    

In [ ]:
# but wait...is it really?
for item in this_cwd.iterdir():
    if item.is_file() and item.suffix != '':
        item_stats = item.stat()
        print(f"File: {item.name}, Suffix: {item.suffix}, Size: {item_stats.st_size} bytes")
    elif item.is_dir():
        print(f"{item.name} is a directory")
    else:
        print(f"{item.name} is not a regular file or directory")

# Entry-level+ ToT

Early papers on ToT were premised on piping requests to the model, typicaly w Python. Some explore CoT as reasonable in a single-sentance prompt. 
Later we find recommendation for "using ToT on tasks requiring deliberate reasoning, on which CoT struggles." [@yao2023tree] [[Yao et al., 2023]](https://arxiv.org/abs/2305.10601)

From ["Tree-of-Thoughts"](http://arxiv.org/abs/2305.10601) by princeton-nlp we get a look at ToT with these more advanced features:
* Adaptive generation of thoughts
* Dynamic stopping criteria
* Support for both breadth-first search (BFS) and depth-first search (DFS) strategies

The [Tweet stream](https://x.com/ShunyuYao12/status/1659357547474681857) is a 1 minute introduction.

In [ ]:
# previously we imported from IPython.display import display, HTML

message_content = message.content

# Extract the text from the TextBlock
text_block = message.content[0]
text = text_block.text
print(text)

## References

```{bibliography}
:style: plain
```